<i><b>Public AI</b></i>
<br>

###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 9. CNN Basis**
# Section 6. CNN 모델 설계 시 고려사항

### _Objective_
1. CNN 모델을 설계할 때에는 한 픽셀 당 얼마만큼의 범위를 처리하고 있는지를 설명하는 Receptive Field의 크기를 계산하는 법을 배웁니다. <br>
2. CNN 모델이 학습 시 얼마만큼의 Memory가 필요한지를 배웁니다. <br>
 

<br><br>

# \[ 1. Receptive Field란? \]

----
----
 
> *Receptive Field는 Input Space에서 얼마 만큼을 계산해 포함했는지에 대한 지표입니다.*<br>
> *Receptive Field가 넓을수록, 더 많은 범위의 정보를 활용해 결과를 계산합니다.*<br>


<br>

## 1. 단일 층일 때의 Receptive Field
---

단일 층일 때의 Receptive Field는 Filter의 크기와 동일합니다.

### (1) Filter Size가 3일 때의 Receptive Field
![Imgur](https://i.imgur.com/ZCMV3DE.png)

### (2) Filter Size가 5일 때의 Receptive Field
![Imgur](https://i.imgur.com/VUlDmQX.png)

<br>
## 2. 복수 층일 때의 Receptive Field
---

복수 층일 경우, stride에 따라 Receptive Field가 크게 달라집니다.

### (1) stride가 1일때의 Receptive Field
![Imgur](https://i.imgur.com/6Wxe00G.png)

### (2) Stride가 2일때의 Receptive Field
![Imgur](https://i.imgur.com/dB8vpmv.png)

### (3) 3층에서의 Recpetive Field
![Imgur](https://i.imgur.com/Gy6Sp2F.png)

<br>

## 3. Receptive Field 계산법
---

Receptive Field를 계산하기 위해서는 아래의 재귀식을 통해 계산할 수 있습니다.<br>

$
r^{[l]} = r^{[l-1]} + (k-1)\times j_{[l-1]} \\
j^{[l]} = j^{[l-1]} \times s^{[l]}\\
------\\
r^{[l]} \mbox{ : receptive field size} \\
j^{[l]} \mbox{ : distance between two points } \\
s^{[l]} \mbox{ : convolution stride size}
$

<br>

## 4. Fully Connected Layer와 Convolution Layer의 비교
---
* 모든 Fully Connected Layer은 Convolution Layer로 바꿀 수 있습니다.<br>
* Fully Connected layer는 Filter Size가 Input size와 동일한 Convolution layer와 동일합니다.

### (1) Fully Connected Layer와 Convolution Layer의 구조도

두 모델은 아래와 같이 도식화할 수 있습니다.

![Imgur](https://i.imgur.com/VGRC6Bf.png)

Convolution Layer의 Filter 크기가 Input Size와 동일하다면, 아래와 같게 됩니다.<br>

![Imgur](https://i.imgur.com/xLLnDhJ.png)

이는 Fully Connected Layer와 Convolution Layer가 사실상 동일한 연산으로 동작합니다.<br>

이 후 많은 모델들이 이러한 특성을 착안해 Fully Convolutional Network,<br>
convolution layer로만 이루어진 모델로 여러 컴퓨터 비전 Task들을 수행합니다.

<br><br>

# \[ 2. CNN 모델의 Memory 요구사항 \]

----
----

> *Convolution Neural Network는 Parameter Sharing을 통해, Parameter의 수를 획기적으로 줄였지만, 학습할 때에는 매우 많은 메모리를 요구합니다.*<br>

<br>

## 1. CNN 모델을 학습할 때 메모리가 많이 필요한 이유
---

* CNN 모델을 학습할 때에는 많은 메모리를 잡아먹습니다.<br>
* CNN 의 sparse 특성은 computer 는
* CNN 모델을 학습할 때 대용량의 GPU 머신이 필요한 이유이기도 합니다.<br>
* 역전파의 수식을 통해 그 이유를 파악해보도록 하겠습니다.


### (1) Weight에 대한 Gradient Descent 

가중치들은 아래의 방식으로 학습됩니다.<br>
$W_{new} = W_{old} - \alpha*\frac{\partial L}{\partial W}$<br>

우리는 위에서 각 가중치들의 기울기인 $\frac{\partial L}{\partial W}$을 구하기 위해서는 <br>
각 층으로 들어온 입력 값($x$)를 기억해야 합니다.

![Imgur](https://i.imgur.com/TTi1Mey.png)

훈련하는 동안에는 Feed Forward로 계산되었던 모든 값이 Back Propagation을 위해<br>
기억해야 합니다. 그래서 적어도 각 층의 계산에서 이용되었던 RAM 양의 전체 합만큼이 필요합니다.

<br>

## 2. 필요 메모리 크기 계산하기
---

LeNet-5에서 이용하는 메모리의 크기를 통해 필요 메모리를 계산해보도록 하겠습니다.<br>

![Imgur](https://i.imgur.com/WCN9wTP.png)


| 층  | 종류 |필터 갯수 | 필터 크기 | 스트라이드 | 패딩 |
|--- |--- | ----|   ----|----|----|
| c1 |합성곱| 6   | (5,5) | 1  | valid |
| s2 |pooling| --- | (2,2) | 2  | valid | 
| c3 |합성곱| 16  | (10,10) | 1 | valid |
| s4 |pooling| --- | (2,2) | 2 | valid |
| c5 |합성곱 | 120  | (5,5) | 1  | valid |
| f6 |FC   | 84   | --- | ---  | --- |
| OUT | softmax | 10 |  --- | --- | --- |



### (1) 전체 파라미터 수 계산하기
각 층별로 필요한 파라미터 수를 계산하면 다음과 같습니다.

| Layer | # Weight | # Bias | # Parameter |
| ----- | ------- | ------- | ------- | 
|  C1   | $(5*5*1*6)=150$|$6$| $150+6=156 $ |
|  S2   | $0$ | $0$ | $0$ |
|  C3   | $(5*5*6*16)=2400$|$16$|$2400+16=2416 $ |
|  S4   | $0$ | $0$ | $0$ |
|  C5   | $(5*5*16*120)=48000$|$120$| $48000+120=48,120$ |
|  F6   | $120*84 = 10,080 $ | $84$ | $ 10,080+84=10164 $
|OUTPUT | $84*10 = 840$ | $10$| $840+10=850$ |

총 파라미터의 수는 61,706개입니다. <br>
각 파라미터 별로 4바이트씩 가진다고 하면, **239KB**정도의 메모리 공간을 차지 합니다.

### (2) 각 층 별 출력 값의 크기 계산하기
각 층별 출력 크기는 아래와 같습니다.

| Layer | FEATURE MAP SHAPE | Total size |
| ----- | -------   | -------   | 
| INPUT | (32,32,1) | 1024 |
|  C1   | (28,28,6) | 4704 |
|  S2   | (14,14,6) | 1176 |
|  C3   | (10,10,16) | 1600 |
|  S4   | (5,5,16) | 400 |
|  C5   | (1,1,120)| 120 |
|  F6   | (84,) | 84 | 
|  OUTPUT   | (10,) | 10 |

총 출력 값의 크기는 9118개입니다.<br>
각 출력 값별로 4바이트씩 가진다고 하면, **36KB**정도의 메모리 공간을 차지합니다.

### (3) 배치 단위 적용하기

실제로 학습을 할 때에는 1개의 데이터가 아닌 복수개의 데이터를 한번에 연산합니다.<br>
배치 단위로 처리하기 때문에, 실제로 필수 메모리를 계산할 때에는 출력 값의 크기에 <br>
배치 크기를 곱해 계산해야 합니다.

배치 단위가 100일때의 필요 메모리 크기 :<br>
$
\mbox{total memory size } = \mbox{parameter size} + \mbox{batch size} * \mbox{feedforward memory size} \\
= 239 + 100 *36 = 3839KB
$

LENET-5와 같이 Input Size가 작은 모델에서는 Memory가 큰 이슈가 되지 않습니다.<br>
하지만 이후 배울 alex net, vggnet, resnet는 수 Gigabytes의 메모리를 필요로 합니다.

### (4) 큰 이미지에서의 메모리 요구사항

ImageNet은 가장 유명한 컴퓨터 비전 데이터셋 중 하나입니다.<br>

![Imgur](https://i.imgur.com/VCT8bo7.png)

위의 데이터 셋을 처리하는 모델들은 보통 (224,224,3)의 크기로 줄이거나 crop해서 이용합니다.<br>

아래의 조건으로 구성된 Convolution Layer가 있습니다.<br>
* filter 갯수 : 256
* filter 크기 : (5,5)
* stride : 1
* padding : SAME

이 층의 출력 값의 크기는 얼마나 될까요? <br>
* output size : $(256,256,256) = 256^3 = 2^{24}$

총 배치가 100 개가 있다면, 차지하는 메모리는 아래와 같습니다.<br>
* memory size : batch size $\times$ output size $\times$ 4bytes
= $100 \times 2^{24} \times 4$ = 6.25GB

보통 큰 단일 GPU 머신이 12GB임을 미루어볼때, CNN은 많은 메모리를 소모하는 것을 <br>
알 수 있습니다.

In [1]:
25*2**24*4/1024/1024/1024 

1.5625

### (5) 메모리를 줄이는 방법, pooling과 Stride

위와 같이 메모리가 가장 많이 먹는 이유 중 하나는 바로, **출력의 크기**에 있습니다.<br>

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---